In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from neurophotonics.fields import *
from neurophotonics.demix import *

Connecting dbadmin@dimitri-proj0.cda95qzjbnvs.us-east-1.rds.amazonaws.com:3306


In [2]:
import datajoint as dj
schema.spawn_missing_classes()

In [3]:
Demix()

design design number,sample,selection selected cells,mix_norm cell's mixing vector norm,demix_norm cell's demixing vector norm,bias_norm cell's bias vector norm,trans_bias_norm don't use. Saved just in case of wrong axis choice,avg_emitter_power (uW) when on
1,0,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,1,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,2,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,3,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,4,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,5,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,6,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,7,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
1,8,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,15030.9
4,0,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,888.889


In [ ]:
dj.U('sample') & Demix()

In [4]:
dj.kill()

  ID USER         HOST          STATE         TIME    INFO
+--+ +----------+ +-----------+ +-----------+ +-----+
   4 rdsadmin     localhost:64094                    5  None
   5 rdsadmin     localhost                       1  None
 691 dbadmin      172.31.89.207:45324                  121  None
 873 dbadmin      172.31.89.207:45710                   55  None
process to kill or "q" to quit > q


In [9]:
x = np.random.randn(1000000,3) 
x /= np.linalg.norm(x, axis=1, keepdims=True)
c = x[:,0]**4
(np.random.rand(c.size) < c).mean()

0.200128

In [ ]:
170*825

In [ ]:
(IlluminationCycle & 'design=4').fetch1('illumination').shape

In [ ]:
(Geometry.Emitter & 'design=3').fetch('e_center_y')

In [ ]:
(Geometry.Detector & 'design=1').fetch('d_center_y').shape

In [ ]:
schema.jobs

In [ ]:
(Geometry.Emitter & 'design=1').fetch('e_center_y').shape

In [ ]:
1296/18

In [ ]:
72/4

In [ ]:
dj.kill()

In [ ]:
Cosine.populate()

In [ ]:
((SpikeSNR() & 'sample=6').fetch1('snr') > 1.0).mean()

In [ ]:
((Cosine() & 'sample=1').fetch1('cosines') > 0.1).mean()

In [ ]:
Demix()

In [ ]:
Sample()

In [ ]:
schema.jobs

In [ ]:
Geometry()

In [ ]:
schema.spawn_missing_classes()

In [ ]:
Tissue()

In [ ]:
dj.config

In [ ]:
dj.config.save_global()

In [ ]:
dj.Diagram(schema)

In [ ]:
keys = DField.fetch('KEY')
fig, ax = plt.subplots(1, len(keys), figsize=(6*len(keys), 6))
for a, key in zip(ax.flatten(), keys):
    (DField & key).plot(axis=a, title=(DSim & key).fetch1('dsim_description'))
fig.savefig('detector-fields.png')

In [ ]:
Tissue.delete()

In [ ]:
EField.populate(reserve_jobs=True)

In [ ]:
keys = EField.fetch('KEY', order_by="KEY")
fig, ax = plt.subplots(1, len(keys), figsize=(6*len(keys), 6))
for a, key in zip(ax.flatten(), keys):
    (EField & key).plot(axis=a, title=(ESim & key).fetch1('esim_description'))
fig.savefig('emitter-fields.png')

In [ ]:
EField.fetc

In [ ]:
!open emitter-fields.pdf

In [ ]:
(EField & 'esim=10').plot()

In [ ]:
ESim.alter()

In [ ]:
ESim.describe()

In [ ]:
DField()

In [ ]:
dj.ERD(schema)

In [ ]:
(DSim() & 'detector_height=10').delete()